In [1]:
#IMPORT
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Set a seed for the random number generator
np.random.seed(100)

In [2]:
## Gaussian Elimination: Scaled Row Pivoting
## This function is based on the pseudo-code on page-148 in the Text by Kincaid and Cheney
def GE_rsp(A):
    '''
    This function returns the P'LU factorization of a square matrix A
    by scaled row partial pivoting. 
    In place of returning L and U, elements of modified A are used to hold values of L and U.
    '''
    m,n = A.shape
    
    L = np.eye(n) # Not being used
    U = np.zeros_like(A) # Not being used
    if m !=n:
        sys.exit("This function needs a square matrix as an input.")
        
    # The initial ordering of rows
    p = list(range(n))
    
    # Scaling vector: absolute maximum elements of each row
    s = np.max(np.abs(A), axis=1)
    
    print("Scaling Vector: ",s)
    
    # Start the k-1 passes of Guassian Elimination on A
    for k in range(n-1):
        
        print("\n PASS {}: \n".format(k+1), A)
        # Find the pivot element and interchange the rows
        pivot_index = k + np.argmax(np.abs(A[p[k:], k])/s[p[k:]])        
        
        # Interchange element in the permutation vector
        if pivot_index !=k:
            temp = p[k]
            p[k]=p[pivot_index]
            p[pivot_index] = temp
            print("permutation vector: ",p)
            
        print("\n Pivot Element: {0:.2f} \n".format(A[p[k],k]))
        if np.abs(A[p[k],k]) < 10**(-20):
             sys.exit("ERROR!! Provided matrix is singular.")
        
        # For the k-th pivot row Perform the Gaussian elimination on the following rows
        for i in range(k+1, n):
            # Find the multiplier
            z = A[p[i],k]/A[p[k],k]
            
            #Save z in A itself. You can save this in L also
            A[p[i],k] = z
            
            #Elimination operation: Changes all elements in a row simultaneously
            ##
            A[p[i],k+1:] -= z*A[p[k],k+1:]
    return A, p

In [3]:
## Example on page number 146 (Kincaid Cheney).
## Example solved manually in class
A = np.array([[2, 3, -6], [1,-6,8], [3, -2, 1]], dtype=float)
print("\n Given A: \n ",A)
A,p =GE_rsp(A)
print("\n After Gaussian Elimination with RSPP: \n", A)
print("\n The permutation Vector is: \n", p)



 Given A: 
  [[ 2.  3. -6.]
 [ 1. -6.  8.]
 [ 3. -2.  1.]]
Scaling Vector:  [6. 8. 3.]

 PASS 1: 
 [[ 2.  3. -6.]
 [ 1. -6.  8.]
 [ 3. -2.  1.]]
permutation vector:  [2, 1, 0]

 Pivot Element: 3.00 


 PASS 2: 
 [[ 0.66666667  4.33333333 -6.66666667]
 [ 0.33333333 -5.33333333  7.66666667]
 [ 3.         -2.          1.        ]]
permutation vector:  [2, 0, 1]

 Pivot Element: 4.33 


 After Gaussian Elimination with RSPP: 
 [[ 0.66666667  4.33333333 -6.66666667]
 [ 0.33333333 -1.23076923 -0.53846154]
 [ 3.         -2.          1.        ]]

 The permutation Vector is: 
 [2, 0, 1]


In [4]:
print("\n Upper triangular, U:\n ", np.triu(A[p,:]))
print("\n Lower triangular, L:\n", np.tril(A[p,:], -1)+np.eye(3))
print("The Permutation Matrix, P:\n",np.eye(3)[p,:])


 Upper triangular, U:
  [[ 3.         -2.          1.        ]
 [ 0.          4.33333333 -6.66666667]
 [ 0.          0.         -0.53846154]]

 Lower triangular, L:
 [[ 1.          0.          0.        ]
 [ 0.66666667  1.          0.        ]
 [ 0.33333333 -1.23076923  1.        ]]
The Permutation Matrix, P:
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [5]:
def back_substitution(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Use backsubstitution to solve the equation Ax=b for 
    upper triangular matrix A (m, n) and vector b (n, 1) 
    """
    n = b.size
    x = np.zeros(n)

    for i in range(n-1, -1, -1):
        for j in range(n-1-i):
            b[i] = b[i] - A[i, n-1-j]*x[n-1-j]
            
        x[i] = b[i]/A[i, i]
            
    return x

def system_solver(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Use row operations to solve the equation Ax=b for 
    upper triangular square matrix A (n, n) and square matrix b (n, n)
    """
    (m, n) = A.shape
    A = np.hstack((A, b))
    for i in range(n-1, -1, -1):
        for j in range(i, -1, -1):
            if (j == i):
                A[i, :] *= 1/A[i, j]
            else:
                A[j, :] -= ((A[j, i]/A[i, i]) * A[i, i]) * A[i, :]
    return A[:, n:]

## Question 1
Modify this code to write a function that solves a linear system Ax =b.  Test this in the case when  𝑏=[3,1,1]^T, and the matrix A = [1 6 0; 2 1 0; 0 2 1]. Only display the solution in the output.

In [6]:
## Gaussian Elimination: Scaled Row Pivoting
## This function is based on the pseudo-code on page-148 in the Text by Kincaid and Cheney
def GE_rsp(A, b):
    '''
    This function returns the P'LU factorization of a square matrix A
    by scaled row partial pivoting. 
    In place of returning L and U, elements of modified A are used to hold values of L and U.
    '''
    # To do this we want to augment the matrix A with the column vector b, the resulting information in the last column 
    # will be our output
    A = np.hstack((A, b))
    m,n = A.shape
    n = n-1
    
    L = np.eye(n) # Not being used
    U = np.zeros_like(A) # Not being used
    if m !=n:
        sys.exit("This function needs a square matrix as an input.")
        
    # The initial ordering of rows
    p = list(range(n))
    
    # Scaling vector: absolute maximum elements of each row
    s = np.max(np.abs(A[:, :-1]), axis=1)
    
#     print("Scaling Vector: ",s)
    print(f"Initial permuation matrix: \n{p}")
#     Start the k-1 passes of Guassian Elimination on A
    for k in range(n-1):
        
        print("\n PASS {}: \n".format(k+1), A)
        # Find the pivot element and interchange the rows
        pivot_index = k + np.argmax(np.abs(A[p[k:], k])/s[p[k:]])        
        
        # Interchange element in the permutation vector
        if pivot_index !=k:
            temp = p[k]
            p[k]=p[pivot_index]
            p[pivot_index] = temp
            print("permutation vector: ",p)
            
        print("\n Pivot Element: {0:.2f} \n".format(A[p[k],k]))
        if np.abs(A[p[k],k]) < 10**(-20):
             sys.exit("ERROR!! Provided matrix is singular.")
        
        # For the k-th pivot row Perform the Gaussian elimination on the following rows
        for i in range(k+1, n):
            # Find the multiplier
            z = A[p[i],k]/A[p[k],k]
            
            #Save z in A itself. You can save this in L also
            A[p[i],k] = z
            
            #Elimination operation: Changes all elements in a row simultaneously
            ##
            A[p[i],k+1:] -= z*A[p[k],k+1:]
        
    # need to do backsubstitution
    U = np.triu(A[p,:-1])
    b = A[p, -1]
    backsub_output = back_substitution(U, b.copy())
        
    return A, p, backsub_output

In [7]:
## Example on page number 146 (Kincaid Cheney).
## Example solved manually in class
A = np.array([[1, 6, 0], [2,1,0], [0, 2, 1]], dtype=float)
# A = np.array([[2, 3, -6], [1,-6,8], [3, -2, 1]], dtype=float)
b = np.array([3, 1, 1], dtype=float).reshape((3, 1))

print("\n Given A: \n ",A)
print(f"Given b: \n{b}")
A,p, x = GE_rsp(A, b)
print("\n After Gaussian Elimination with RSPP: \n", A)
print("\n The permutation Vector is: \n", p)
print(f"The answer to the equation is:\n x =  {x}")


 Given A: 
  [[1. 6. 0.]
 [2. 1. 0.]
 [0. 2. 1.]]
Given b: 
[[3.]
 [1.]
 [1.]]
Initial permuation matrix: 
[0, 1, 2]

 PASS 1: 
 [[1. 6. 0. 3.]
 [2. 1. 0. 1.]
 [0. 2. 1. 1.]]
permutation vector:  [1, 0, 2]

 Pivot Element: 2.00 


 PASS 2: 
 [[0.5 5.5 0.  2.5]
 [2.  1.  0.  1. ]
 [0.  2.  1.  1. ]]
permutation vector:  [1, 2, 0]

 Pivot Element: 2.00 


 After Gaussian Elimination with RSPP: 
 [[ 0.5   2.75 -2.75 -0.25]
 [ 2.    1.    0.    1.  ]
 [ 0.    2.    1.    1.  ]]

 The permutation Vector is: 
 [1, 2, 0]
The answer to the equation is:
 x =  [0.27272727 0.45454545 0.09090909]


## Question 2
Modify this code to find the determinant of any square matrix A. Note that PA = LU implies that determinant of A  = plus or minus determinant of U. The sign depends of the number of row-swaps in the elimination process.  Use this code to find the determinant of any 10x10 matrix that you randomly generate. Compare your result with the built-in NumPy method. 

In [8]:
## Gaussian Elimination: Scaled Row Pivoting
## This function is based on the pseudo-code on page-148 in the Text by Kincaid and Cheney
def GE_rsp(A):
    '''
    This function returns the P'LU factorization of a square matrix A
    by scaled row partial pivoting. 
    In place of returning L and U, elements of modified A are used to hold values of L and U.
    '''
    m,n = A.shape
    
    L = np.eye(n) # Not being used
    U = np.zeros_like(A) # Not being used
    if m !=n:
        sys.exit("This function needs a square matrix as an input.")
        
    # The initial ordering of rows
    p = list(range(n))
    
    # Scaling vector: absolute maximum elements of each row
    s = np.max(np.abs(A), axis=1)
    
#     print("Scaling Vector: ",s)
    
    num_swaps = 0
    # Start the k-1 passes of Guassian Elimination on A
    for k in range(n-1):
        
#         print("\n PASS {}: \n".format(k+1), A)
        # Find the pivot element and interchange the rows
        pivot_index = k + np.argmax(np.abs(A[p[k:], k])/s[p[k:]])        
        
        # Interchange element in the permutation vector
        if pivot_index !=k:
            temp = p[k]
            p[k]=p[pivot_index]
            p[pivot_index] = temp
            num_swaps += 1
#             print("permutation vector: ",p)
            
#         print("\n Pivot Element: {0:.2f} \n".format(A[p[k],k]))
        if np.abs(A[p[k],k]) < 10**(-20):
             sys.exit("ERROR!! Provided matrix is singular.")
        
        # For the k-th pivot row Perform the Gaussian elimination on the following rows
        for i in range(k+1, n):
            # Find the multiplier
            z = A[p[i],k]/A[p[k],k]
            
            #Save z in A itself. You can save this in L also
            A[p[i],k] = z
            
            #Elimination operation: Changes all elements in a row simultaneously
            ##
            A[p[i],k+1:] -= z*A[p[k],k+1:]
            
        # Det U = PI[Uii]
        # Det A = Det U * (-1)^num_swaps
        det = 1
        upper = np.triu(A[p,:])
        for i in range(n):
            det *= upper[i, i]
        det *= (-1)**num_swaps
            
        
    return A, p, det

In [9]:
# Now with a random 10x10 matrix 
A_orig = np.random.rand(10,10) * 8
A_orig = np.ceil(A_orig)

In [10]:
print("\n Given A: \n ",A_orig)
A, p, det = GE_rsp(A_orig.copy())
# print("\n After Gaussian Elimination with RSPP: \n", A)
print("\n The permutation Vector is: \n", p)
print(f"The determinant is: {det}")
print(f"The numpy determined determinant is: {np.linalg.det(A_orig)}")


 Given A: 
  [[5. 3. 4. 7. 1. 1. 6. 7. 2. 5.]
 [8. 2. 2. 1. 2. 8. 7. 2. 7. 3.]
 [4. 8. 7. 3. 2. 3. 1. 3. 7. 1.]
 [5. 5. 1. 4. 1. 8. 8. 1. 8. 5.]
 [6. 6. 5. 1. 2. 5. 7. 3. 3. 7.]
 [8. 8. 3. 5. 3. 3. 2. 2. 1. 5.]
 [4. 5. 6. 2. 8. 8. 5. 4. 3. 2.]
 [3. 2. 2. 8. 8. 5. 6. 3. 1. 4.]
 [5. 1. 5. 8. 4. 3. 7. 7. 3. 6.]
 [5. 3. 1. 3. 4. 6. 3. 6. 2. 6.]]

 The permutation Vector is: 
 [1, 2, 5, 8, 6, 3, 4, 9, 7, 0]
The determinant is: 27226009.99999999
The numpy determined determinant is: 27226009.999999978


## Question 3
Modify the system-solver that you have created to find the inverse of a square matrix. Use this code to display the inverse of  A = [1 6 0; 2 1 0; 0 2 1].

In [11]:
## Gaussian Elimination: Scaled Row Pivoting
## This function is based on the pseudo-code on page-148 in the Text by Kincaid and Cheney
def GE_rsp(A):
    '''
    This function returns the P'LU factorization of a square matrix A
    by scaled row partial pivoting. 
    In place of returning L and U, elements of modified A are used to hold values of L and U.
    '''
    # To solve for the inverse, augment the original matrix A with the identity matrix and perform gausian elimination
    # as normal
    m,n = A.shape
    
    # check this first 
    if m !=n:
        sys.exit("This function needs a square matrix as an input.")
        
        
    L = np.eye(n) # Not being used
    U = np.zeros_like(A) # Not being used
    
    augment = np.eye(n)
    A = np.concatenate((A, augment), axis=1)
        
    # The initial ordering of rows
    p = list(range(n))
    
    # Scaling vector: absolute maximum elements of each row
    s = np.max(np.abs(A), axis=1)
    
    print("Scaling Vector: ",s)
    
    # Start the k-1 passes of Guassian Elimination on A
    for k in range(n-1):
        
#         print("\n PASS {}: \n".format(k+1), A)
        # Find the pivot element and interchange the rows
        pivot_index = k + np.argmax(np.abs(A[p[k:], k])/s[p[k:]])        
        
        # Interchange element in the permutation vector
        if pivot_index !=k:
            temp = p[k]
            p[k]=p[pivot_index]
            p[pivot_index] = temp
#             print("permutation vector: ",p)
            
#         print("\n Pivot Element: {0:.2f} \n".format(A[p[k],k]))
        if np.abs(A[p[k],k]) < 10**(-20):
             sys.exit("ERROR!! Provided matrix is singular.")
        
        # For the k-th pivot row Perform the Gaussian elimination on the following rows
        for i in range(k+1, n):
            # Find the multiplier
            z = A[p[i],k]/A[p[k],k]
            
            #Save z in A itself. You can save this in L also
            A[p[i],k] = z
            
            #Elimination operation: Changes all elements in a row simultaneously
            ##
            A[p[i],k+1:] -= z*A[p[k],k+1:]
            
    U = np.triu(A[p,:-n])
    b = A[p, -n:]

    solver_output = system_solver(U, b.copy())

    return A, p, solver_output

In [12]:
## Example on page number 146 (Kincaid Cheney).
## Example solved manually in class
# A_orig = np.array([[2, 3, -6], [1,-6,8], [3, -2, 1]], dtype=float)
A_orig = np.array([[1, 6, 0], [2,1,0], [0, 2, 1]], dtype=float)
print("\n Given A: \n ",A_orig)



 Given A: 
  [[1. 6. 0.]
 [2. 1. 0.]
 [0. 2. 1.]]


In [13]:
A, p, A_inv =GE_rsp(A_orig.copy())
print("\n After Gaussian Elimination with RSPP: \n", A)
print("\n The permutation Vector is: \n", p)
print(f"\n A inverse is: \n{A_inv}\n")
print(f"A * A_inv is: \n{np.matmul(A_orig, A_inv)}\n")
print(f"A_inv * A is: \n{np.matmul(A_inv, A_orig)}\n")

Scaling Vector:  [6. 2. 2.]

 After Gaussian Elimination with RSPP: 
 [[ 0.5   2.75 -2.75  1.   -0.5  -2.75]
 [ 2.    1.    0.    0.    1.    0.  ]
 [ 0.    2.    1.    0.    0.    1.  ]]

 The permutation Vector is: 
 [1, 2, 0]

 A inverse is: 
[[-0.09090909  0.54545455  0.        ]
 [ 0.18181818 -0.09090909  0.        ]
 [-0.36363636  0.18181818  1.        ]]

A * A_inv is: 
[[ 1.00000000e+00 -5.55111512e-17  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]

A_inv * A is: 
[[1.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.22044605e-16 1.00000000e+00]]



## Iterative Methods for Linear Systems
---


### Jacobi Method

In [21]:
# You can modify this code to answer the following
'''
Jacobi's iteration method for solving the system of equations Ax=b.
p0 is the initialization for the iteration.
'''
def jacobi(A, b, p0, tol, maxIter=100):
    n=len(A)
    p = p0

    for k in range(maxIter):
        p_old = p.copy() # In python assignment is not the same as copy
        
        # Update every component of iterant p
        for i in range(n):
            sumi = b[i];
            for j in range(n):
                if i==j: # Diagonal elements are not included in Jacobi
                    continue;
                sumi = sumi - A[i,j] * p_old[j]
            p[i] = sumi/A[i,i]
                
        rel_error = np.linalg.norm(p-p_old)/n
        print("Relative error in iteration", k+1,":",rel_error)
        if rel_error<tol:
            print("TOLERANCE MET BEFORE MAX-ITERATION")
            break
    return p;

def gauss_seidel(A, b, p0, tol, maxIter=100):
    n=len(A)
    p = p0

    for k in range(maxIter):
        p_old = p.copy() # In python assignment is not the same as copy
        
        # Update every component of iterant p
        for i in range(n):
            sumi = b[i];
            for j in range(i-1):
                sumi = sumi - A[i, j] * p[j]
            for h in range(i+1, n):
                sumi = sumi - A[i, h] * p_old[h]
            p[i] = sumi/A[i,i]
                
        rel_error = np.linalg.norm(p-p_old)/n
        print("Relative error in iteration", k+1,":",rel_error)
        print(A)
        if rel_error<tol:
            print("TOLERANCE MET BEFORE MAX-ITERATION")
            break
    return p;

In [23]:
# Example System
A = np.array([[10, -1, 2, 0],
              [-1, 11, -1, 3],
              [2, -1, 10, -1],
              [0, 3, -1, 8]],dtype=float)
b = np.array([6, 25, -11, 15],dtype=float)

A = np.array([])

In [24]:
gauss_seidel(A.copy(), b.copy(), 4*np.random.randn(4), .000001)

Relative error in iteration 1 : 3.1680030869226776
[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]
Relative error in iteration 2 : 0.5874561697381738
[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]
Relative error in iteration 3 : 0.03653497028502936
[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]
Relative error in iteration 4 : 0.0062733088406822304
[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]
Relative error in iteration 5 : 0.0007252748214516372
[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]
Relative error in iteration 6 : 5.3105456728702656e-05
[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]
Relative error in iteration 7 : 3.017486918967517e-06
[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]
Relative error in iteration 8 : 4.878541545052466e-07
[[10. -1.  2.  0.]
 [-1. 11. -1.  3.

array([ 1.02136279,  1.84193969, -1.18584526,  1.18427262])

In [17]:
gauss_seidel(A.copy(), b.copy(), 4*np.random.randn(4), .000001)

Relative error in iteration 1 : 0.7831755233065237
Relative error in iteration 2 : 0.14412897212226108
Relative error in iteration 3 : 0.016064321321412313
Relative error in iteration 4 : 0.002359924491624639
Relative error in iteration 5 : 0.00021835890710485053
Relative error in iteration 6 : 1.27576043182693e-05
Relative error in iteration 7 : 1.1960871193038494e-06
Relative error in iteration 8 : 1.9400653176506072e-07
TOLERANCE MET BEFORE MAX-ITERATION


array([ 1.02136296,  1.84193972, -1.18584532,  1.18427261])